In [3]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline,FunctionTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import make_column_transformer, make_column_selector

In [4]:
dev_data = pd.read_csv('dataset/dev_data.csv', index_col=['src','slice_num'])
dev_data.head()

img_slice  \
src slice_num                                              
0   1          og_dataset/splits/dev/slice_0_img_001.npy   
    12         og_dataset/splits/dev/slice_0_img_012.npy   
    66         og_dataset/splits/dev/slice_0_img_066.npy   
    88         og_dataset/splits/dev/slice_0_img_088.npy   
2   47         og_dataset/splits/dev/slice_2_img_047.npy   

                                               mask_slice  mask_mean_0  \
src slice_num                                                            
0   1          og_dataset/splits/dev/slice_0_mask_001.npy     0.000000   
    12         og_dataset/splits/dev/slice_0_mask_012.npy     0.000000   
    66         og_dataset/splits/dev/slice_0_mask_066.npy     0.000000   
    88         og_dataset/splits/dev/slice_0_mask_088.npy     0.000000   
2   47         og_dataset/splits/dev/slice_2_mask_047.npy     0.246052   

               mask_mean_1  mask_mean_2    img_mean            lng  \
src slice_num                                                        
0   1             0.000000     0.000000  133.604416  348977.894737   
    12            0.000000     0.000000  233.865952  515979.122807   
    66            0.000000     0.000000  188.873245  424887.543860   
    88            0.000000     0.000000  290.440277  531161.052632   
2   47            0.225334     0.020718  493.879822  195829.787435   

                        lat  \
src slice_num                 
0   1          3.572176e+06   
    12         3.572176e+06   
    66         3.632905e+06   
    88         3.648087e+06   
2   47         3.460402e+06   

                                                         np_mask  debris_mean  \
src slice_num                                                                   
0   1          [[[0 0 0]\n  [0 0 0]\n  [0 0 0]\n  ...\n  [0 0...     0.000000   
    12         [[[0 0 0]\n  [0 0 0]\n  [0 0 0]\n  ...\n  [0 0...     0.000000   
    66         [[[0 0 0]\n  [0 0 0]\n  [0 0 0]\n  ...\n  [0 0...     0.000000   
    88         [[[0 0 0]\n  [0 0 0]\n  [0 0 0]\n  ...\n  [0 0...     0.000000   
2   47         [[[0 0 1]\n  [0 0 1]\n  [0 0 1]\n  ...\n  [0 0...     0.020718   

               clean_mean  glacier_perc  label  
src slice_num                                   
0   1            0.000000      1.000000      1  
    12           0.000000      1.000000      1  
    66           0.000000      1.000000      1  
    88           0.000000      1.000000      1  
2   47           0.225334      0.246052      1

In [5]:
def load(file):
    try:
        return np.load(file)
    except:
        return pd.NA
    
dev_data['features'] = dev_data.img_slice.apply(lambda x: load(x))
dev_data.head()

img_slice  \
src slice_num                                              
0   1          og_dataset/splits/dev/slice_0_img_001.npy   
    12         og_dataset/splits/dev/slice_0_img_012.npy   
    66         og_dataset/splits/dev/slice_0_img_066.npy   
    88         og_dataset/splits/dev/slice_0_img_088.npy   
2   47         og_dataset/splits/dev/slice_2_img_047.npy   

                                               mask_slice  mask_mean_0  \
src slice_num                                                            
0   1          og_dataset/splits/dev/slice_0_mask_001.npy     0.000000   
    12         og_dataset/splits/dev/slice_0_mask_012.npy     0.000000   
    66         og_dataset/splits/dev/slice_0_mask_066.npy     0.000000   
    88         og_dataset/splits/dev/slice_0_mask_088.npy     0.000000   
2   47         og_dataset/splits/dev/slice_2_mask_047.npy     0.246052   

               mask_mean_1  mask_mean_2    img_mean            lng  \
src slice_num                                                        
0   1             0.000000     0.000000  133.604416  348977.894737   
    12            0.000000     0.000000  233.865952  515979.122807   
    66            0.000000     0.000000  188.873245  424887.543860   
    88            0.000000     0.000000  290.440277  531161.052632   
2   47            0.225334     0.020718  493.879822  195829.787435   

                        lat  \
src slice_num                 
0   1          3.572176e+06   
    12         3.572176e+06   
    66         3.632905e+06   
    88         3.648087e+06   
2   47         3.460402e+06   

                                                         np_mask  debris_mean  \
src slice_num                                                                   
0   1          [[[0 0 0]\n  [0 0 0]\n  [0 0 0]\n  ...\n  [0 0...     0.000000   
    12         [[[0 0 0]\n  [0 0 0]\n  [0 0 0]\n  ...\n  [0 0...     0.000000   
    66         [[[0 0 0]\n  [0 0 0]\n  [0 0 0]\n  ...\n  [0 0...     0.000000   
    88         [[[0 0 0]\n  [0 0 0]\n  [0 0 0]\n  ...\n  [0 0...     0.000000   
2   47         [[[0 0 1]\n  [0 0 1]\n  [0 0 1]\n  ...\n  [0 0...     0.020718   

               clean_mean  glacier_perc  label  \
src slice_num                                    
0   1            0.000000      1.000000      1   
    12           0.000000      1.000000      1   
    66           0.000000      1.000000      1   
    88           0.000000      1.000000      1   
2   47           0.225334      0.246052      1   

                                                        features  
src slice_num                                                     
0   1          [[[nan, nan, nan, nan, nan, nan, nan, nan, nan...  
    12         [[[nan, nan, nan, nan, nan, nan, nan, nan, nan...  
    66         [[[58.0, 45.0, 41.0, 49.0, 59.0, 126.0, 141.0,...  
    88         [[[54.0, 46.0, 45.0, 38.0, 77.0, 118.0, 128.0,...  
2   47         [[[-1.7699648, -1.5762353, -1.6766024, -1.5867...

In [6]:
from sklearn import set_config
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder
set_config(display="diagram")

make_pipeline(KBinsDiscretizer(), OneHotEncoder())

Pipeline(steps=[('kbinsdiscretizer', KBinsDiscretizer()),
                ('onehotencoder', OneHotEncoder())])

In [39]:
feature_names = ['LE7 B1 (blue)', 'LE7 B2 (green)', 'LE7 B3 (red)', 'LE7 B4 (near infrared)', 'LE7 B5 (shortwave infrared 1)', 'LE7 B6_VCID_1 (low-gain thermal infrared)', 'LE7 B6_VCID_2 (high-gain thermal infrared)', 'LE7 B7 (shortwave infrared 2)', 'LE7 B8 (panchromatic)', 'LE7 BQA (quality bitmask)', 'NDVI (vegetation index)', 'NDSI (snow index)', 'NDWI (water index)', 'SRTM 90 elevation', 'SRTM 90 slope']

def split(list_matrix, columns = []):
    df = list_matrix.apply(
        lambda m: [m[0][:,:,i].flat for i in range(m[0].shape[2])], 
        axis = 1,
        result_type='expand')
    if len(columns) == len(df.columns):
        df.columns = columns
    return df

splitter = FunctionTransformer(func=split, kw_args= {'columns':feature_names})

(110, 15)

In [48]:
from sklearn.base import BaseEstimator,TransformerMixin
from collections import defaultdict

class ColorHistogram(BaseEstimator,TransformerMixin):
    def __init__(self, binning='doane'):
        self.bins = dict()
        self.n_features = None
        
        self.binning_param = binning
        self.binning = binning
            
    def fit(self, X, y=None):
        self.n_features = X.shape[1]
        
        if type(self.binning) is not dict:
            self.binning = defaultdict(lambda: self.binning_param)

        for col_name in X.columns:
            # accumulate all values
            values = np.hstack(X[col_name])
            # remove NaN
            values = values[np.logical_not(np.isnan(values))]
            
            # compute the bins using the choosed binning method
            bins = np.histogram_bin_edges(values, bins=self.binning[col_name])
            self.bins[col_name] =  bins
        return self
    
    def transform(self, X, y=None):
        histos = dict()
        for col_name in X.columns:
            # select each column 
            C = np.stack(X[col_name])
            # apply the histogram function for each row
            hist = np.apply_along_axis(
                ColorHistogram.histogram,
                axis = 1,
                arr = C,
                bins = self.bins[col_name]
            )
            histos[col_name] = hist 
        return np.hstack(list(histos.values()))
    
    @staticmethod
    def histogram(slice, bins, density=True):
        return np.histogram(slice, bins, density=density)[0]

# ColorHistogram().fit_transform(c)

/home/mamiglia/.local/lib/python3.10/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


In [ ]:
pipe = make_column_transformer(
    (make_pipeline(
        splitter,
        ColorHistogram()
    ), ['features']),
)

histograms = pipe.fit_transform(dev_data)
histograms

In [82]:
print('Bin size for each column')
fitted_bins = pipe.transformers_[0][1].steps[1][1].bins
for col, bin in fitted_bins.items():
    print(f'{col}: {len(bin)}')

Bin size for each column
LE7 B1 (blue): 39
LE7 B2 (green): 39
LE7 B3 (red): 39
LE7 B4 (near infrared): 39
LE7 B5 (shortwave infrared 1): 39
LE7 B6_VCID_1 (low-gain thermal infrared): 38
LE7 B6_VCID_2 (high-gain thermal infrared): 38
LE7 B7 (shortwave infrared 2): 39
LE7 B8 (panchromatic): 39
LE7 BQA (quality bitmask): 39
NDVI (vegetation index): 36
NDSI (snow index): 37
NDWI (water index): 37
SRTM 90 elevation: 38
SRTM 90 slope: 38


In [83]:
bin_lens = {col:len(bin)-1 for col, bin in fitted_bins.items()}
new_cols = [(col, i) for col, length in bin_lens.items() for i in range(length)]
d_hist = pd.DataFrame(a, columns=pd.MultiIndex.from_tuples(new_cols))
d_hist.head()

LE7 B1 (blue)                                                        ...  \
             0    1    2    3    4        5         6    7    8    9   ...   
0           NaN  NaN  NaN  NaN  NaN      NaN       NaN  NaN  NaN  NaN  ...   
1           NaN  NaN  NaN  NaN  NaN      NaN       NaN  NaN  NaN  NaN  ...   
2           0.0  0.0  0.0  0.0  0.0  0.00000  0.000000  0.0  0.0  0.0  ...   
3           0.0  0.0  0.0  0.0  0.0  0.00000  0.000000  0.0  0.0  0.0  ...   
4           0.0  0.0  0.0  0.0  0.0  0.03399  0.080618  0.0  0.0  0.0  ...   

  SRTM 90 slope                                                              
             27        28        29        30   31   32   33   34   35   36  
0      0.000053  0.000108  0.000021  0.000034  0.0  0.0  0.0  0.0  0.0  0.0  
1      0.000334  0.000132  0.000072  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  
2      0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  
3      0.000051  0.000026  0.000018  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  
4      0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 559 columns]

0       0.469123
1       0.469123
2      16.054648
3      16.054649
4      16.054649
         ...    
105     0.000000
106     0.000000
107     0.000000
108     0.000000
109     0.469123
Length: 110, dtype: float64